# Mohammadhossein Amini
## **"K-State Honor Code "On my honor, as a student, I have neither given nor received unauthorized aid on this academic work.**

# Part II

### In the previois part, I considered the global sale for movies since some movies didn't have any sale in US. In that case, I could have more movies to analyse but the accuracy of the model was missleading. For this part, I decided to select only domestic sales and therefore, I had to drop some more movies. Also, I create a new model since the original Y for training set is different. 

In [62]:
import pandas as pd
import re
import json
import numpy as np
from pattern.en import sentiment
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
#regression packages
import sklearn.linear_model as lm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
import statsmodels.formula.api as sm
from statsmodels.regression.linear_model import RegressionResults
#lasso regression
from sklearn import linear_model

In [33]:
#Loading new dataset (Second week)
# create an empty list to store our tweets in
data = []

# append each line of the data to our tweets list using the json module
for line in open('Movie(New).json'):
    try:
        data.append(json.loads(line))
    except:
        pass

# lets see how many we got
print len(data)

53177


In [34]:
#There are 29 movies in the data set and the hashtags used to extract them are as follows:
HashtagsUsed=['#Wonderstruck', '#TheSnowman', '#samekindofdifferentasme', '#geostorm', '#boo2', '#OnlytheBrave', '#TheKillingofaSacredDeer', '#LEATHERFACE', '#Novitiate', '#Suburbicon', '#TheWork', '#junglemovie', '#BadDayForTheCut', '#TragedyGirls', '#KillingGunther', '#neverhere', '#KillsOnWheels', '#TheBachelors', '#samuraijack', '#theforeigner', '#happydeathday', '#americansatan', '#BreatheMovie', '#overdrivemovie', '#bloodmoneymovie', '#GnomeAlone', '#GoodbyeChristopherRobin', '#Marshallmovie', '#ProfessorMarston']
ListOfMovies=['Wonderstruck', 'TheSnowman', 'samekindofdifferentasme', 'geostorm', 'boo2', 'OnlytheBrave', 'TheKillingofaSacredDeer', 'LEATHERFACE', 'Novitiate', 'Suburbicon', 'TheWork', 'junglemovie', 'BadDayForTheCut', 'TragedyGirls', 'KillingGunther', 'neverhere', 'KillsOnWheels', 'TheBachelors', 'samuraijack', 'theforeigner', 'happydeathday', 'americansatan', 'BreatheMovie', 'overdrivemovie', 'bloodmoneymovie', 'GnomeAlone', 'GoodbyeChristopherRobin', 'Marshallmovie', 'ProfessorMarston']

### 1.a.i

In [35]:
#To filter out the error messages
texts = [ T['text'] for T in data if 'text' in T ]

In [36]:
tweets = []
for T in data:
    if 'text' in T:
        tweets.append(T)
len(tweets)

53170

### 1.a.ii

In [37]:
print "there are %s tweets in the dataset" %(len(texts))

there are 53170 tweets in the dataset


### 1.a.iii

In [38]:
screen_names = [T['user']['screen_name'] for T in tweets]
from collections import Counter

c = Counter(screen_names)
print "there are %s unique users in the dataset" %(len(c))

there are 30659 unique users in the dataset


### 1.a.iv

In [39]:
originaltweets = []
retweets_only = []
for tweet in texts:
    if 'rt @' not in tweet.lower():
        originaltweets.append(tweet)
    if 'rt @' in tweet.lower():
        retweets_only.append(tweet)
print "the number of original tweets is: %s" %(len(originaltweets))
print "the number of retweets is: %s" %(len(retweets_only))
print "the ratio of original tweets and retweets is: %s " %(np.true_divide(len(originaltweets),len(retweets_only)))

the number of original tweets is: 20489
the number of retweets is: 32681
the ratio of original tweets and retweets is: 0.626939200147 


In [40]:
FinalTable=[]
for index in range(0,len(ListOfMovies)):
    Indtweets=[]
    for tweet in tweets:
        if ListOfMovies[index] in re.findall(r"(?<=#)\w+", str(tweet)):
            Indtweets.append(tweet)
    texts = [ T['text'] for T in Indtweets if 'text' in T ]
    NubmerOfTweets=len(Indtweets)
    screen_names = [T['user']['screen_name'] for T in Indtweets]
    c = Counter(screen_names)
    NumberOfUniqueUsers=len(c)
    retweets_only = []
    for tweet in texts:
        if 'rt @' in tweet.lower():
            retweets_only.append(tweet)
    NumberOfRetweets=len(retweets_only)
    followers_count = [status['user']['followers_count'] for status in Indtweets if 'user' in status]
    SumOfFollowers=0
    for i in followers_count:
        SumOfFollowers=SumOfFollowers+i
    friends_count = [status['user']['friends_count'] for status in Indtweets if 'user' in status]
    SumOfFriends=0
    for i in friends_count:
        SumOfFriends=SumOfFriends+i
    
    text=[]
    for tweet in Indtweets:
        if 'rt @' not in tweet['text'].lower():
            text.append(tweet['text'])
    # Cleaning the text
    texts_clean_completely = []
    for i in text:
        result = ' '.join(re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", i).split())
        texts_clean_completely.append(result)
    score=0
    for sentence in texts_clean_completely:
        score = score + sentiment(sentence)[0]
    FinalTable.append([ListOfMovies[index],NubmerOfTweets,NumberOfUniqueUsers,NumberOfRetweets,SumOfFollowers,SumOfFriends,score])
df=pd.DataFrame(FinalTable)     

In [41]:
df = df.rename(columns={0: 'Name_Movie', 1: 'Nubmer_Tweets', 2:'Number_Unique_Users', 3:'Number_Retweets',4: 'Sum_Followers', 5:'Sum_Friends', 6:'Tweets_Sentiment_Score'})
df.head()

,Name_Movie,Nubmer_Tweets,Number_Unique_Users,Number_Retweets,Sum_Followers,Sum_Friends,Tweets_Sentiment_Score
0,Wonderstruck,660,402,363,11613705,1140556,36.477260
1,TheSnowman,3073,2049,1587,31872631,7449731,332.564819
2,samekindofdifferentasme,61,47,23,306804,334786,11.622197
3,geostorm,994,829,308,3246310,1703415,91.005918
4,boo2,428,264,307,2441599,296235,15.792634


In [42]:
y=[219179, 30579084, 4776427, 137117136,35574251,12727152,423166,792146,0,3125536,18007,21452,1509,30254,0,0,38515,0,0,126725299,68511730,194554,190717,3590166,0,0, 4057578,7056165 ,1528258]
df['Sales']=y
df.head()

,Name_Movie,Nubmer_Tweets,Number_Unique_Users,Number_Retweets,Sum_Followers,Sum_Friends,Tweets_Sentiment_Score,Sales
0,Wonderstruck,660,402,363,11613705,1140556,36.477260,219179
1,TheSnowman,3073,2049,1587,31872631,7449731,332.564819,30579084
2,samekindofdifferentasme,61,47,23,306804,334786,11.622197,4776427
3,geostorm,994,829,308,3246310,1703415,91.005918,137117136
4,boo2,428,264,307,2441599,296235,15.792634,35574251


In [46]:
df

,Name_Movie,Nubmer_Tweets,Number_Unique_Users,Number_Retweets,Sum_Followers,Sum_Friends,Tweets_Sentiment_Score,Sales
0,Wonderstruck,660,402,363,11613705,1140556,36.477260,219179
1,TheSnowman,3073,2049,1587,31872631,7449731,332.564819,30579084
2,samekindofdifferentasme,61,47,23,306804,334786,11.622197,4776427
3,geostorm,994,829,308,3246310,1703415,91.005918,137117136
4,boo2,428,264,307,2441599,296235,15.792634,35574251
5,OnlytheBrave,266,222,132,12718936,881802,25.067389,12727152
6,TheKillingofaSacredDeer,244,205,182,2566245,354477,2.979545,423166
7,LEATHERFACE,192,161,79,260293,281151,4.316748,792146
8,Novitiate,1187,417,810,16261711,1091697,63.795175,0
9,Suburbicon,7742,5767,5629,185750669,16185209,156.199130,3125536


### 1.b.i, 1.b.ii

In [49]:
# drop the movies that we have dropped in the previous model!.
df=df.drop(df.index[[8,9,14,15,16,17,18,24,25]])
df.to_csv('df_test_DroppedZeros.csv', index=False)
df

,Name_Movie,Nubmer_Tweets,Number_Unique_Users,Number_Retweets,Sum_Followers,Sum_Friends,Tweets_Sentiment_Score,Sales
0,Wonderstruck,660,402,363,11613705,1140556,36.477260,219179
1,TheSnowman,3073,2049,1587,31872631,7449731,332.564819,30579084
2,samekindofdifferentasme,61,47,23,306804,334786,11.622197,4776427
3,geostorm,994,829,308,3246310,1703415,91.005918,137117136
4,boo2,428,264,307,2441599,296235,15.792634,35574251
5,OnlytheBrave,266,222,132,12718936,881802,25.067389,12727152
6,TheKillingofaSacredDeer,244,205,182,2566245,354477,2.979545,423166
7,LEATHERFACE,192,161,79,260293,281151,4.316748,792146
10,TheWork,82,78,30,923553,96412,6.533333,18007
11,junglemovie,4,4,0,890,664,0.569697,21452


In [89]:
# Here, to be consistent, I drop LEATHERFACE, junglemovie, BadDayForTheCut, and overdrivemovie as well.
df=df.drop(df.index[[7,9,10,16]])
df

,Name_Movie,Nubmer_Tweets,Number_Unique_Users,Number_Retweets,Sum_Followers,Sum_Friends,Tweets_Sentiment_Score,Sales
0,Wonderstruck,660,402,363,11613705,1140556,36.477260,132065
1,TheSnowman,3073,2049,1587,31872631,7449731,332.564819,1212950
2,samekindofdifferentasme,61,47,23,306804,334786,11.622197,1252142
3,geostorm,994,829,308,3246310,1703415,91.005918,5903428
4,boo2,428,264,307,2441599,296235,15.792634,10052608
5,OnlytheBrave,266,222,132,12718936,881802,25.067389,3502095
6,TheKillingofaSacredDeer,244,205,182,2566245,354477,2.979545,225171
8,TheWork,82,78,30,923553,96412,6.533333,3286
11,TragedyGirls,462,256,341,5326639,628795,16.229196,12432
12,theforeigner,470,438,308,535103,809248,26.422960,3443433


### 1.b.iii.

In [92]:
# Top 5 movies in terns of # of tweets.
df.sort_values(['Nubmer_Tweets'], ascending=False).head()

,Name_Movie,Nubmer_Tweets,Number_Unique_Users,Number_Retweets,Sum_Followers,Sum_Friends,Tweets_Sentiment_Score,Sales
1,TheSnowman,3073,2049,1587,31872631,7449731,332.564819,1212950
3,geostorm,994,829,308,3246310,1703415,91.005918,5903428
13,happydeathday,686,588,114,1601058,695955,107.995052,5078685
0,Wonderstruck,660,402,363,11613705,1140556,36.477260,132065
17,GoodbyeChristopherRobin,658,468,244,4188622,2635522,160.720180,337302


### 1.b.iv.

In [93]:
# Top 5 movies in terms of # of unique users.
df.sort_values(['Number_Unique_Users'], ascending=False).head()

,Name_Movie,Nubmer_Tweets,Number_Unique_Users,Number_Retweets,Sum_Followers,Sum_Friends,Tweets_Sentiment_Score,Sales
1,TheSnowman,3073,2049,1587,31872631,7449731,332.564819,1212950
3,geostorm,994,829,308,3246310,1703415,91.005918,5903428
13,happydeathday,686,588,114,1601058,695955,107.995052,5078685
15,BreatheMovie,622,515,590,12800330,471140,3.780808,56322
17,GoodbyeChristopherRobin,658,468,244,4188622,2635522,160.720180,337302


### 1.b.v.

In [94]:
# Top 5 movies in terms of # movie sales.
df.sort_values(['Sales'], ascending=False).head()

,Name_Movie,Nubmer_Tweets,Number_Unique_Users,Number_Retweets,Sum_Followers,Sum_Friends,Tweets_Sentiment_Score,Sales
4,boo2,428,264,307,2441599,296235,15.792634,10052608
3,geostorm,994,829,308,3246310,1703415,91.005918,5903428
13,happydeathday,686,588,114,1601058,695955,107.995052,5078685
5,OnlytheBrave,266,222,132,12718936,881802,25.067389,3502095
12,theforeigner,470,438,308,535103,809248,26.422960,3443433


In [63]:
# Reading the training dataset saved before
df_tr=pd.read_csv('df_DroppedZeros.csv')
df_tr.head()

,Name_Movie,Nubmer_Tweets,Number_Unique_Users,Number_Retweets,Sum_Followers,Sum_Friends,Tweets_Sentiment_Score,Sales
0,Wonderstruck,1732,1213,1135,39773353,2380837,57.999895,65882
1,TheSnowman,10387,6985,6427,104240086,24462028,410.567675,3372565
2,samekindofdifferentasme,115,87,32,519068,111138,16.328409,2591985
3,geostorm,1367,1130,296,6595985,1972428,161.369466,13707376
4,boo2,1314,1154,940,12717463,1409507,62.072978,21226953


#### Here, I drop LEATHERFACE, junglemovie, BadDayForTheCut, and overdrivemovie since I don't have any sale for current and next week sale.

In [66]:
df_tr=df_tr.drop(df_tr.index[[7,9,10,16]])

In [85]:
#Generating the randomforestregressor model with trainig set
from sklearn.ensemble import RandomForestRegressor as RFR
RFR_model=RFR(n_estimators=100, criterion='mse')
RFR_model.fit(X, y)
RFR_y = RFR_model.predict(X)
print "mean square error: ", mean_squared_error(y, RFR_y)
print "variance or r-squared: ", explained_variance_score(y, RFR_y)

mean square error:  2.47077761702e+14
variance or r-squared:  0.852018606416


### 1.c.i., 1.c.ii

In [58]:
DiffDf=df[['Name_Movie','Nubmer_Tweets']]
DiffDf = DiffDf.rename(columns={'Nubmer_Tweets': 'Nubmer_Tweets(2ndWeek)'})
DiffDf['Nubmer_Tweets(1stWeek)']=df_tr[['Nubmer_Tweets']]
DiffDf['Growth']=DiffDf['Nubmer_Tweets(2ndWeek)']-DiffDf['Nubmer_Tweets(1stWeek)']
DiffDf.sort(['Growth'], ascending=False).head()

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Name_Movie,Nubmer_Tweets(2ndWeek),Nubmer_Tweets(1stWeek),Growth
6,TheKillingofaSacredDeer,244,106.0,138.0
10,TheWork,82,44.0,38.0
2,samekindofdifferentasme,61,115.0,-54.0
19,theforeigner,470,625.0,-155.0
13,TragedyGirls,462,800.0,-338.0


## 2

In [95]:
#assigning columns to X and Y variables
y = df['Sales'] 
X = df.drop(['Sales','Name_Movie'], axis =1)

In [96]:
df.drop(['Name_Movie'], axis =1).corr()

,Nubmer_Tweets,Number_Unique_Users,Number_Retweets,Sum_Followers,Sum_Friends,Tweets_Sentiment_Score,Sales
Nubmer_Tweets,1.000000,0.991977,0.952573,0.859569,0.959486,0.921736,0.084618
Number_Unique_Users,0.991977,1.000000,0.931078,0.827497,0.939745,0.916152,0.124044
Number_Retweets,0.952573,0.931078,1.000000,0.908884,0.898788,0.799092,-0.001639
Sum_Followers,0.859569,0.827497,0.908884,1.000000,0.840070,0.722336,-0.115071
Sum_Friends,0.959486,0.939745,0.898788,0.840070,1.000000,0.960674,-0.029998
Tweets_Sentiment_Score,0.921736,0.916152,0.799092,0.722336,0.960674,1.000000,0.050764
Sales,0.084618,0.124044,-0.001639,-0.115071,-0.029998,0.050764,1.000000


In [100]:
#running model with randomforestregressor
RFR_y = RFR_model.predict(X)
print "mean square error: ", mean_squared_error(y, RFR_y)

mean square error:  1.76037223553e+15


In [109]:
#Asked MSE in the PDF file: (Part 2.c.)
print "Asked MSE is equal to: %s million" %(np.mean(abs(y-RFR_y))/1000000)

Asked MSE is equal to: 27.8302061719 million
